In [8]:
import numpy as np
import csv
import plotly
import plotly.graph_objs as go
from PIL import Image, ImageDraw, ImageFont
from scipy.ndimage import gaussian_filter, gaussian_filter1d
from math import asin, acos, degrees, sin, cos, pi
from scipy.optimize import least_squares
from filterpy.kalman import KalmanFilter
polyval2d = np.polynomial.polynomial.polyval2d

plotly.offline.init_notebook_mode(connected=True)
py = plotly.offline

# read data

filename = 'rssi_train.csv'

def convert(row):
    id = row[0]
    rssi = row[1]
    x = row[2]
    y = row[3]
    return (id, float(rssi), (float(x) + 390) / 3, (float(y) + 960) / 3)

beacon_ids = ['cd', 'd7', '17', '51', '43', 'b8', '2a', 'f8', '3d', '62',
            '3']
    
def read_data():
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        data = map(convert, reader)

    data_for_beacon = {}
    for id in beacon_ids:
        data_for_beacon[id] = []
        
    for row in data:
        beacon = row[0]
        data_for_beacon[beacon].append(row)

    return data_for_beacon

data_for_beacon = read_data()

pixel_origin = np.array([125, 410])
global_origin = np.array([55.94455753546212,-3.1866420060396194])

coor_change = np.array([[-2.62187e6, 422601.], [-799717., -1.41637e6]])

# convert global coordinates to pixels

beacons_global = np.array([
    [55.9444578385393,-3.1866151839494705],
    [55.94444244275808,-3.18672649562358860],
    [55.94452336441765,-3.1866540759801865],
    [55.94452261340533,-3.1867526471614838],
    [55.94448393625199,-3.1868280842900276],
    [55.94449050761571,-3.1866483762860294],
    [55.94443774892113,-3.1867992505431175],
    [55.944432116316044,-3.186904862523079],
    [55.94444938963575,-3.1869836524128914],
    [55.94449107087541,-3.186941407620907]
])

beacons_pixel = np.array(np.transpose(coor_change.dot(np.transpose(beacons_global - global_origin))) + pixel_origin, dtype=np.int16)

def rssi_from_data(data):
    return np.array([row[1] for row in data], dtype=np.float)

def pos_from_data(data):
    return np.array([[row[2], row[3]] for row in data], dtype=np.float)

def dist_from_point(point, positions):
    diff = positions - point
    return np.linalg.norm(diff, axis=1)

class Beacon(object):
    def __init__(self, b_id, pos):
        self.name = b_id
        self.pos = pos
        self.points = pos_from_data(data_for_beacon[b_id])
        self.rssi = rssi_from_data(data_for_beacon[b_id])
        self.dist = dist_from_point(pos, self.points)
        self.f_dist_rssi = np.poly1d(np.polyfit(self.rssi, self.dist, 2))
        self.f_rssi_dist = np.poly1d(np.polyfit(self.dist, self.rssi, 2))
        
    def norm(self, point, axis=None):
        return np.linalg.norm(point - self.pos, axis=axis)
        
b_17 = Beacon('17', beacons_pixel[2])
b_b8 = Beacon('b8', beacons_pixel[5])

def gamma(a, b, c):
    cosval = (a ** 2 + b ** 2 - c ** 2)/(2.0 * a * b)
    if cosval > 1.:
        cosval = 1.
    elif cosval < -1.:
        cosval = -1.
    return acos(cosval)

v = b_b8.pos - b_17.pos
r = np.linalg.norm(v)
alpha = asin(v[1] / v[0])

def get_position_from_dist(arg):   
    r_17, r_b8 = arg
    beta = gamma(r_17, r, r_b8)
    return b_17.pos + [r_17 * cos(alpha + beta), r_17 * sin(alpha + beta)]

def read_data_17_b8():
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        rssi_17_b8 = []
        positions_17_b8 = []
        last_17 = -70
        last_b8 = -70
        for [b_id, rssi, x, y] in reader:
            rssi, x, y = int(rssi), (int(x) + 390.) / 3, (int(y) + 960.) / 3
            if b_id == '17':
                last_17 = rssi
                rssi_17_b8.append(np.array([last_17, last_b8], dtype=np.float))
                positions_17_b8.append(np.array([x, y], dtype=np.float))
            elif b_id == 'b8':
                last_b8 = rssi
                rssi_17_b8.append(np.array([last_17, last_b8], dtype=np.float))
                positions_17_b8.append(np.array([x, y], dtype=np.float))

    return (np.array(rssi_17_b8, dtype=np.float), np.array(positions_17_b8, dtype=np.float))

rssi_17_b8, positions_17_b8 = read_data_17_b8()

dist_from_17 = b_17.norm(positions_17_b8, axis=1)
dist_from_b8 = b_b8.norm(positions_17_b8, axis=1)

# def get_rel_dist_from_17_b8(rssi):
#     return np.array([pred, )])

def get_pos_from_rssi(rssi):
    return get_position_from_dist(*get_rel_dist_from_17_b8(rssi))

def get_rssi_from_pos(pos):
    return np.array([b_17.f_rssi_dist(b_17.norm(pos)), b_b8.f_rssi_dist(b_b8.norm(pos))])

def estimate_position_from_rssi(rssi):
#     smooth_rssi = gaussian_filter1d(rssi, sigma=16, axis=0)
    return np.apply_along_axis(get_pos_from_rssi, 1, rssi)

def estimate_position_from_dist(dist):
    return np.apply_along_axis(get_position_from_dist, 1, dist)

def estimate_rssi(pos):
    return np.apply_along_axis(get_rssi_from_pos, 1, pos)

In [37]:
# drawing images
    
def draw_image(points, b):
    # get an image
    img = Image.open('map.png')
    fnt = ImageFont.truetype('/Users/piotr/Library/Fonts/Literation Mono Powerline.ttf', 40)
    
    d = ImageDraw.Draw(img)

    d.text((10,10), b, font=fnt, fill=(0,0,0,255))

    for (ident, beacon) in zip(beacon_ids, beacons_pixel):
        (x, y) = beacon
        d.text((x-5,y-5), ident, font=fnt, fill=(0,0,0,255))
#         d.ellipse([x-2, y-2, x+2, y+2], outline=(0,0,0,0), fill=(255,255,255,0))

    for point in points:
        x = point[0]
        y = point[1]
        d.ellipse([x-2, y-2, x+2, y+2], outline=(50,50,50,255), fill=(50,50,50,255))

#     Image.composite(img, Image.new('RGB', img.size, 'white'), img)
    img.save("{}_read.png".format(b),"PNG")
    
draw_image(b_b8.points, 'b8')

In [14]:
# RSSI along different radii

endpoints = np.concatenate([
    np.stack([np.linspace(125, 480, 6), np.full(6, 1040)], axis=1), 
    np.array([[x, y] for x in [125, 480] for y in [1040 - 60, 1040 - 120]])
    ])

def plot_points_on_line(points, line):
    # get an image
    img = Image.open('map.png')
    fnt = ImageFont.truetype('/Users/piotr/Library/Fonts/Literation Mono Powerline.ttf', 40)
    
    d = ImageDraw.Draw(img)

    for point in points:
        x = point[0]
        y = point[1]
        d.ellipse([x-2, y-2, x+2, y+2], outline=(255,0,0,0), fill=(255,0,0,0))
        
    for point in line:
        x = point[0]
        y = point[1]
        d.ellipse([x-2, y-2, x+2, y+2], outline=(0,255,0,0), fill=(0,255,0,0))

    img.show()
    
def dist_from_line(points, start, end):
    return np.absolute(np.cross(end-start,points-start) / np.linalg.norm(end - start))
    
    
def map_points_on_line(points, start, end):
    line = np.stack([np.linspace(start[0], end[0]), np.linspace(start[1], end[1])], axis=1)
    margin = dist_from_line(points, start, end)
    plot_points_on_line(points_b8[margin < 20], line)

    
def plot_rssi_dist(rssi, dist, title):
    f = bestfit_rssi_dist(rssi, dist)
    domain = np.linspace(0, 650)
    py.iplot({
        "data": [
            go.Scatter(x=dist, y=rssi, mode='markers'),
            go.Scatter(x=domain, y=f(domain))
        ],
        "layout": go.Layout(title=title)
    }) 
    

def plot_rssi_dist_on_line(points, rssi, start, end):
    dist = dist_from_point(points, start)
    margin = dist_from_line(points, start, end)
    plot_rssi_dist(rssi[margin < 20], dist[margin < 20], title=end)
        
def bestfit_rssi_dist(rssi, dist):
    return np.poly1d(np.polyfit(dist, rssi, 2))
    
def plot_all(points, rssi, start, beacon):
    dist = dist_from_point(points, start)
    domain = np.linspace(0, 650)
    fitlines = []
    ends = []
    for end in endpoints:
        margin = dist_from_line(points, start, end)
        line_dist = dist[margin < 20]
        line_rssi = rssi[margin < 20]
        f = bestfit_rssi_dist(line_rssi, line_dist)
        fitlines.append(f)
        ends.append(end)

#         py.iplot({
#             "data": [go.Scatter(x=line_dist, y=line_rssi, mode='markers')] + [go.Scatter(x=domain, y=f(domain))],
#             "layout": go.Layout(title=repr(list(end)))
#         }) 
        
    py.iplot({
        "data": [go.Scatter(x=dist, y=rssi, mode='markers')] + [go.Scatter(x=domain, y=f(domain), name=repr(list(end))) for (f, end) in zip(fitlines, ends)],
        "layout": go.Layout(title=beacon)
    }) 

plot_all(b_b8.points, b_b8.rssi, b_b8.pos, b_b8.name)

In [21]:
# dist = f(rssi)

def f_dist_rssi_poly2d(params):
    return polyval2d(rssi_17_b8[:, 0], rssi_17_b8[:, 1], np.reshape(params, (2, 3)))

def fun_17(params):
    return dist_from_17 - f_dist_rssi_poly2d(params)

def fun_b8(params):
    return dist_from_b8 - f_dist_rssi_poly2d(params)

x0 = np.random.normal(size=6)
# x0 = np.zeros(6)

pred_dist_from_17 = f_dist_rssi_poly2d(least_squares(fun_17, x0).x)
pred_dist_from_b8 = f_dist_rssi_poly2d(least_squares(fun_b8, x0).x)

pred_dist = np.stack([pred_dist_from_17, pred_dist_from_b8], axis=1)

pred_dist_from_17_sv = b_17.f_dist_rssi(rssi_17_b8[:, 0])
pred_dist_from_b8_sv = b_b8.f_dist_rssi(rssi_17_b8[:, 1])


def plot_dist_actual_and_predicted(actual, predicted, predicted_sv, name, start, n):
    stop = start + n
    domain = np.linspace(0, 1, n)
    
    py.iplot({
        "data": [
            go.Scatter(x=domain, y=actual[start:stop], name='actual'),
            go.Scatter(x=domain, y=predicted[start:stop], name='predicted with signals from b8 and 17'),
            go.Scatter(x=domain, y=predicted_sv[start:stop], name='predicted with signal from self')
        ],
        "layout": go.Layout(
            title=name,
            yaxis=dict(
                range=[-50, 800]
            ),
            xaxis=dict(
                showticklabels=False
            )
        )
    })


plot_dist_actual_and_predicted(dist_from_17, pred_dist_from_17, pred_dist_from_17_sv, '17', 0, 1000)
plot_dist_actual_and_predicted(dist_from_b8, pred_dist_from_b8, pred_dist_from_b8_sv, 'b8', 0, 1000)


def error(actual, predicted):
    return np.sqrt(np.sum((actual - predicted) ** 2) / actual.size)

(error(dist_from_17, pred_dist_from_17), 
 error(dist_from_17, pred_dist_from_17_sv), 
 error(dist_from_b8, pred_dist_from_b8), 
 error(dist_from_b8, pred_dist_from_b8_sv))

(99.733158978217617,
 134.49550846277077,
 106.33553334304268,
 116.45997376249045)

In [ ]:
# rssi kalman filter

rssi_kf = KalmanFilter(2, 2)
rssi_kf.x = np.array([-70, -70])
rssi_kf.P *= 400
rssi_kf.R = r_cov
rssi_kf.Q = np.eye(2) * 4  # TODO need to test on data with walking speed
rssi_kf.H = np.eye(2)

kalman_rssi = rssi_kf.batch_filter(rssi_17_b8)[0]

def plot_rssi_actual_and_predicted(actual, predicted, start, n):
    stop = start + n
    domain = np.linspace(0, 1, n)
    
    def plot(i, name):
        py.iplot({
            "data": [
                go.Scatter(x=domain, y=actual[start:stop, i], name='actual'),
                go.Scatter(x=domain, y=predicted[start:stop, i], name='predicted')
            ],
            "layout": go.Layout(
                title=name,
                yaxis=dict(
                    range=[-100, -50]
                )
            )
        })
        
    plot(0, '17')
    plot(1, 'b8')
    
plot_rssi_actual_and_predicted(rssi_17_b8, kalman_rssi, 0, 3100)

In [ ]:
estimated_position = estimate_position_from_dist(pred_dist)

pos_kf = KalmanFilter(2, 2)
pos_kf.x = np.array([250, 500])
pos_kf.P = np.diag([150 ** 2, 350 ** 2])
pos_kf.R = np.cov(estimated_position.T - positions_17_b8.T)
pos_kf.Q = np.eye(2) * 16  # TODO need to test on data with walking speed
pos_kf.H = np.eye(2)

kalman_pos = pos_kf.batch_filter(estimated_position)[0]

In [ ]:
def plot_predictions(actual, estimated, start, n):
    stop = start + n
#     estimated_position = np.apply_along_axis(get_pos_from_rssi, 1, rssi)
    
    def plot(i, name):
        py.iplot({
            "data": [
                go.Scatter(x=np.linspace(0, 100, n), y=actual[start:stop,i]),
                go.Scatter(x=np.linspace(0, 100, n), y=estimated[start:stop,i])
            ],
            "layout": go.Layout(
                title=name
            )
        })
        
    plot(0, 'x')
    plot(1, 'y')
    
plot_predictions(positions_17_b8, kalman_pos, 0, 3100)

In [ ]:
1